In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install sentence-transformers
!pip install pinecone

### Config things

In [3]:
import os

# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Строим путь на три уровня вверх
config_path = os.path.join(current_dir, '..', '..', 'llama.config')

# Нормализуем путь (убираем лишние '..' и т.д.)
config_path = os.path.normpath(config_path)


# Читаем файл
with open(config_path, 'r') as file:
    api_base = file.readline().strip()
    my_model = file.readline().strip()
    API_KEY = file.readline().strip()
    YOUR_API_KEY = file.readline().strip()
#    index_name = file.readline().strip()
index_name = 'red-llama'

# Выводим значения
#print(f"API Base: {api_base}")
#print(f"My Model: {my_model}")
#print(f"API Key: {API_KEY}")
print(f"Pinecone API Key: {YOUR_API_KEY}")
print(f"Pinecone index name: {index_name}")


Pinecone API Key: pcsk_2MHdUF_M8e8hZxWQN2NFvQ4CWhXvsyL7EwPzNpvmy7iYRCN8hpey5u1TvqHZJ89TjJmZUx
Pinecone index name: red-llama


In [5]:

from tqdm import tqdm
import pandas as pd

# Отключаем прогресс-бары для pandas
tqdm.pandas(disable=True)

In [15]:
import os
from pinecone import Pinecone

# Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY, environment="us-east1-gcp")  # Укажите окружение

# Имя индекса
index_name = "red-llama"

# Проверка существования индекса
if index_name in pc.list_indexes().names():
    print(f"Индекс '{index_name}' существует.")
    
    # Подключение к индексу
    index = pc.Index(index_name)
    
    # Получение статистики индекса
    index_stats = index.describe_index_stats()
    print("Статистика индекса:", index_stats)
    
    # Пример запроса данных (если нужно)
    # query_response = index.query(
    #     vector=[0.1] * 384,  # Пример вектора для запроса
    #     top_k=5,  # Количество ближайших векторов
    #     include_values=True  # Включить значения векторов в ответ
    # )
    # print("Результаты запроса:", query_response)
else:
    print(f"Индекс '{index_name}' не существует.")


Индекс 'red-llama' существует.
Статистика индекса: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 557}},
 'total_vector_count': 557,
 'vector_type': 'dense'}


## Загрузка данных

## Извлечение данных с Pinecone

In [8]:
import json
from pinecone import Pinecone


# 1. Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY, environment="us-east1-gcp")  # Укажите окружение

# 2. Подключение к индексу
#index_name = "your-index-name"  # Замените на имя вашего индекса
index = pc.Index(index_name)



# 3. Генерация списка всех ID
start_id = 0
end_id = 567
all_ids = [f"chunk_{i}" for i in range(start_id, end_id + 1)]

# 4. Извлечение данных с помощью метода fetch (один запрос)
try:
    fetch_response = index.fetch(ids=all_ids)
    print(f"Извлечено {len(fetch_response.vectors)} векторов из {len(all_ids)} запрошенных ID.")
except Exception as e:
    print(f"Ошибка при извлечении данных: {e}")
    exit()

# 5. Обработка данных
vectors = []
for vector_id, vector_data in fetch_response.vectors.items():
    vectors.append({
        "id": vector_id,
        "vector": vector_data.values,  # Значения вектора
        "metadata": vector_data.metadata  # Метаданные
    })

# 6. Сохранение данных в JSON файл
output_file = "pinecone_fetch_data.json"
with open(output_file, "w") as f:
    json.dump(vectors, f, indent=4)

print(f"Данные успешно сохранены в файл {output_file}")
print(f"Всего извлечено векторов: {len(vectors)}")


Извлечено 556 векторов из 568 запрошенных ID.
Данные успешно сохранены в файл pinecone_fetch_data.json
Всего извлечено векторов: 556


In [9]:
import pandas as pd
import json

# 1. Загрузка данных из JSON файла
output_file = "pinecone_fetch_data.json"
with open(output_file, "r") as f:
    vectors = json.load(f)

# 2. Преобразование в DataFrame
df = pd.DataFrame(vectors)

# 3. Вывод первых нескольких строк DataFrame
print(df.head())


          id                                             vector  \
0  chunk_338  [-0.0225879718, 0.0184257105, -0.0145661086, -...   
1  chunk_161  [0.025534546, 0.0278284252, -0.0194324423, 0.0...   
2  chunk_277  [-0.0209172536, 0.0307390876, -0.0386536568, -...   
3  chunk_175  [0.0294892043, -0.0505454838, 0.0347524546, -0...   
4  chunk_160  [-0.0361339115, -0.0174505413, -0.0816515312, ...   

                                            metadata  
0  {'answer': 'По нормативным значениям АК (автом...  
1  {'answer': 'Это инъекции, которые разглаживают...  
2  {'answer': 'Первый ресторан в России со звездо...  
3  {'answer': 'The Telegram development team is b...  
4  {'answer': 'Начинайте с низкой концентрации и ...  


In [ ]:
import pandas as pd

'''
# Пример DataFrame
data = {
    "id": ["chunk_338", "chunk_161", "chunk_277", "chunk_175"],
    "vector": [
        [-0.0225879718, 0.0184257105, -0.0145661086],
        [0.025534546, 0.0278284252, -0.0194324423],
        [-0.0209172536, 0.0307390876, -0.0386536568],
        [0.0294892043, -0.0505454838, 0.0347524546]
    ]
}
df = pd.DataFrame(data)
'''
# 1. Создайте список всех ожидаемых ID
start_id = 0
end_id = 567
all_ids = [f"chunk_{i}" for i in range(start_id, end_id + 1)]

# 2. Извлеките список ID из DataFrame
present_ids = df["id"].tolist()

# 3. Найдите отсутствующие ID
missing_ids = set(all_ids) - set(present_ids)

# 4. Выведите результат
print(f"Отсутствующие ID: {sorted(missing_ids)}")




Отсутствующие ID: ['chunk_380', 'chunk_381', 'chunk_382', 'chunk_383', 'chunk_384', 'chunk_385', 'chunk_386', 'chunk_387', 'chunk_388', 'chunk_389', 'chunk_390', 'chunk_391']


Отсутствующие ID: ['chunk_380', 'chunk_381', 'chunk_382', 'chunk_383', 'chunk_384', 'chunk_385', 'chunk_386', 'chunk_387', 'chunk_388', 'chunk_389', 'chunk_390', 'chunk_391']


Вектор chunk_407 содержит только нули и будет удален.
Вектор chunk_557 содержит только нули и будет удален.
Вектор chunk_444 содержит только нули и будет удален.
Вектор chunk_394 содержит только нули и будет удален.
Вектор chunk_397 содержит только нули и будет удален.
Вектор chunk_415 содержит только нули и будет удален.
Вектор chunk_546 содержит только нули и будет удален.
Вектор chunk_405 содержит только нули и будет удален.
Вектор chunk_455 содержит только нули и будет удален.
Вектор chunk_410 содержит только нули и будет удален.
Загружено 50 векторов
Загружено 100 векторов
Загружено 150 векторов
Загружено 200 векторов
Загружено 250 векторов
Загружено 300 векторов
Загружено 350 векторов
Загружено 400 векторов
Загружено 450 векторов
Загружено 500 векторов
Загружено 546 векторов
Данные успешно загружены в Pinecone!

In [11]:
df.info

<bound method DataFrame.info of             id                                             vector  \
0    chunk_338  [-0.0225879718, 0.0184257105, -0.0145661086, -...   
1    chunk_161  [0.025534546, 0.0278284252, -0.0194324423, 0.0...   
2    chunk_277  [-0.0209172536, 0.0307390876, -0.0386536568, -...   
3    chunk_175  [0.0294892043, -0.0505454838, 0.0347524546, -0...   
4    chunk_160  [-0.0361339115, -0.0174505413, -0.0816515312, ...   
..         ...                                                ...   
551  chunk_554  [-0.00727135316, -0.0198816322, -0.0630982593,...   
552  chunk_245  [-0.00947351288, 0.0156911984, -0.00623399531,...   
553  chunk_286  [0.024843974, 0.00707773, -0.0402551107, 0.005...   
554  chunk_105  [-0.00596816931, 0.0651620179, 0.0414247811, 0...   
555  chunk_337  [0.0343624614, 0.0673869103, 0.0294149406, -0....   

                                              metadata  
0    {'answer': 'По нормативным значениям АК (автом...  
1    {'answer': 'Это инъе

In [12]:
import pandas as pd

# Пример DataFrame
data = {
    "id": ["chunk_338", "chunk_161", "chunk_277", "chunk_175"],
    "vector": [
        [-0.0225879718, 0.0184257105, -0.0145661086],
        [0.025534546, 0.0278284252, -0.0194324423],
        [-0.0209172536, 0.0307390876, -0.0386536568],
        [0.0294892043, -0.0505454838, 0.0347524546]
    ],
    "metadata": [
        {"chunk_text": "Text 1", "other_key": "value1"},
        {"chunk_text": "Text 2", "other_key": "value2"},
        {"chunk_text": "Text 3", "other_key": "value3"},
        {"chunk_text": "Text 4", "other_key": "value4"}
    ]
}
#df = pd.DataFrame(data)

# Извлечение chunk_text из metadata
df["chunk_text"] = df["metadata"].apply(lambda x: x.get("chunk_text"))

# Вывод первых строк DataFrame
print(df.head())


          id                                             vector  \
0  chunk_338  [-0.0225879718, 0.0184257105, -0.0145661086, -...   
1  chunk_161  [0.025534546, 0.0278284252, -0.0194324423, 0.0...   
2  chunk_277  [-0.0209172536, 0.0307390876, -0.0386536568, -...   
3  chunk_175  [0.0294892043, -0.0505454838, 0.0347524546, -0...   
4  chunk_160  [-0.0361339115, -0.0174505413, -0.0816515312, ...   

                                            metadata  \
0  {'answer': 'По нормативным значениям АК (автом...   
1  {'answer': 'Это инъекции, которые разглаживают...   
2  {'answer': 'Первый ресторан в России со звездо...   
3  {'answer': 'The Telegram development team is b...   
4  {'answer': 'Начинайте с низкой концентрации и ...   

                                          chunk_text  
0  9. Как рассчитываются нагрузки от транспортных...  
1  Что такое ботокс для лица? Это инъекции, котор...  
2  Какой ресторан и когда в России получил первую...  
3  Where is Telegram based? The Telegram

### Подготовка чанков

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример DataFrame
data = {
    "id": ["chunk_1", "chunk_2", "chunk_3", "chunk_4"],
    "chunk_text": [
        "This is the first document.",
        "This document is the second document.",
        "And this is the third one.",
        "Is this the first document?"
    ]
}
#df = pd.DataFrame(data)

# 1. Инициализация TfidfVectorizer
vectorizer = TfidfVectorizer()

# 2. Преобразование текстов в TF-IDF матрицу
tfidf_matrix = vectorizer.fit_transform(df["chunk_text"])

# 3. Преобразование матрицы в DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# 4. Добавление ID к результату
tfidf_df.insert(0, "id", df["id"])
tfidf_df.insert(1, "chunk_text", df["chunk_text"])

# Вывод результата
print(tfidf_df)



            id                                         chunk_text   00  000  \
0    chunk_338  9. Как рассчитываются нагрузки от транспортных...  0.0  0.0   
1    chunk_161  Что такое ботокс для лица? Это инъекции, котор...  0.0  0.0   
2    chunk_277  Какой ресторан и когда в России получил первую...  0.0  0.0   
3    chunk_175  Where is Telegram based? The Telegram developm...  0.0  0.0   
4    chunk_160  Как использовать ретинол? Начинайте с низкой к...  0.0  0.0   
..         ...                                                ...  ...  ...   
551  chunk_554  \nWas the Group's revolving credit facility (R...  0.0  0.0   
552  chunk_245  Чем отличается мобильное приложение World Clas...  0.0  0.0   
553  chunk_286  7. Как оформляется испытательный срок при прие...  0.0  0.0   
554  chunk_105  Как французский бульдог относится к одиночеств...  0.0  0.0   
555  chunk_337  8. Какие требования к габаритам мостов указаны...  0.0  0.0   

     000kg  000oz  000to  001  003  004  ...  являю

##  Загрузка векторов на Pinecone

In [22]:
print(tfidf_df.shape)  # (количество строк, количество столбцов)


(556, 8074)


In [26]:
from pinecone import Pinecone
import pandas as pd

# 1. Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY, environment="us-east1-gcp")  # Укажите окружение

tf_idf_index_name = "red-llama-tf-idf"

# 2. Подключение к индексу
index = pc.Index(tf_idf_index_name)
# 384
# 8074 - 2

# 2. Подготовка данных
# Предположим, что tfidf_df содержит столбцы "id", "chunk_text" и TF-IDF векторы
vectors = []
for _, row in tfidf_df.iterrows():
    vector_id = row["id"]  # Уникальный идентификатор
    vector_values = row.drop(["id", "chunk_text"]).tolist()  # TF-IDF вектор
    metadata = {"chunk_text": row["chunk_text"]}  # Метаданные (например, текст)
    vectors.append((vector_id, vector_values, metadata))

# 3. Удаление нулевых векторов
filtered_vectors = []
for vector_id, vector_values, metadata in vectors:
    if any(vector_values):  # Проверка, есть ли хотя бы одно ненулевое значение
        filtered_vectors.append((vector_id, vector_values, metadata))
    else:
        print(f"Вектор {vector_id} содержит только нули и будет удален.")

# 4. Загрузка данных по пакетам
batch_size = 50  # Уменьшите размер пакета
for i in range(0, len(filtered_vectors), batch_size):
    batch = filtered_vectors[i:i + batch_size]
    index.upsert(batch)
    print(f"Загружено {i + len(batch)} векторов")

# 3. Загрузка данных в Pinecone
# index.upsert(vectors)

print("Данные успешно загружены в Pinecone!")



Вектор chunk_407 содержит только нули и будет удален.
Вектор chunk_557 содержит только нули и будет удален.
Вектор chunk_444 содержит только нули и будет удален.
Вектор chunk_394 содержит только нули и будет удален.
Вектор chunk_397 содержит только нули и будет удален.
Вектор chunk_415 содержит только нули и будет удален.
Вектор chunk_546 содержит только нули и будет удален.
Вектор chunk_405 содержит только нули и будет удален.
Вектор chunk_455 содержит только нули и будет удален.
Вектор chunk_410 содержит только нули и будет удален.
Загружено 50 векторов
Загружено 100 векторов
Загружено 150 векторов
Загружено 200 векторов
Загружено 250 векторов
Загружено 300 векторов
Загружено 350 векторов
Загружено 400 векторов
Загружено 450 векторов
Загружено 500 векторов
Загружено 546 векторов
Данные успешно загружены в Pinecone!


### Сохранение первых записей в JSON

In [20]:
# Сохранение первых записей в JSON
tfidf_df.head(1).to_json("tfidf_vectors.json", orient="records", lines=True)


## Обучение и сохранение векторизатора TfidfVectorizer

In [ ]:
from pinecone import Pinecone
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle


tf_idf_index_name = "red-llama-tf-idf"

# 1. Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY)
index = pc.Index(tf_idf_index_name)


# 3. Обучите TfidfVectorizer на ваших данных

corpus = df["chunk_text"].tolist()

# Обучение TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)  # Обучаем вектор на корпусе текстов


# Сохранение обученного вектора
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("Обученный TfidfVectorizer сохранен.")








Обученный TfidfVectorizer сохранен.
Загруженный TfidfVectorizer: TfidfVectorizer()
ID: chunk_547
Сходство: 0.862579823
Текст: What was Brave Bison's total revenue for 2022?
£31.7 million

---
ID: chunk_552
Сходство: 0.346977443
Текст: 
What was the total number of monthly views generated by Brave Bison Media Network?
Over 1 billion monthly views

---
ID: chunk_551
Сходство: 0.326618254
Текст: 
What percentage of Brave Bison staff work on a hybrid basis?
Over 50%

---
ID: chunk_565
Сходство: 0.316588402
Текст: 

8. What was the total assets as of June 30, 2022?
Answer: $22,929,553


---
ID: chunk_560
Сходство: 0.298785508
Текст: 

3. What was the net profit for 2022?
Answer: $491,955


---


## Загрузка векторизатора TfidfVectorizer

In [41]:
# 2. Подготовка вектора запроса
query_text = "What was Brave Bison's total revenue for 2022?"  # Замените на ваш текст


# Загрузка обученного TfidfVectorizer
with open("tfidf_vectorizer.pkl", "rb") as f:
    loaded_vectorizer = pickle.load(f)


vectorizer = loaded_vectorizer

print("Загруженный TfidfVectorizer:", loaded_vectorizer)

# 4. Преобразуйте текст запроса в вектор
query_vector = vectorizer.transform([query_text]).toarray().flatten().tolist()

# 5. Выполнение поиска
results = index.query(
    vector=query_vector,  # Вектор запроса
    top_k=5,  # Количество ближайших векторов
    include_metadata=True  # Включить метаданные в результаты
)

# 6. Обработка результатов
for match in results["matches"]:
    print(f"ID: {match['id']}")
    print(f"Сходство: {match['score']}")
    print(f"Текст: {match['metadata']['chunk_text']}")
    print("---")

Загруженный TfidfVectorizer: TfidfVectorizer()
ID: chunk_547
Сходство: 0.862579823
Текст: What was Brave Bison's total revenue for 2022?
£31.7 million

---
ID: chunk_552
Сходство: 0.346977443
Текст: 
What was the total number of monthly views generated by Brave Bison Media Network?
Over 1 billion monthly views

---
ID: chunk_551
Сходство: 0.326618254
Текст: 
What percentage of Brave Bison staff work on a hybrid basis?
Over 50%

---
ID: chunk_565
Сходство: 0.316588402
Текст: 

8. What was the total assets as of June 30, 2022?
Answer: $22,929,553


---
ID: chunk_560
Сходство: 0.298785508
Текст: 

3. What was the net profit for 2022?
Answer: $491,955


---


### Q_A_rectifier_technologies.txt

## Q_A_Brave_Bison.txt

In [42]:
# 2. Подготовка вектора запроса
query_text = "Starvest plc?"  # Замените на ваш текст




# 4. Преобразуйте текст запроса в вектор
query_vector = vectorizer.transform([query_text]).toarray().flatten().tolist()

# 5. Выполнение поиска
results = index.query(
    vector=query_vector,  # Вектор запроса
    top_k=5,  # Количество ближайших векторов
    include_metadata=True  # Включить метаданные в результаты
)

# 6. Обработка результатов
for match in results["matches"]:
    print(f"ID: {match['id']}")
    print(f"Сходство: {match['score']}")
    print(f"Текст: {match['metadata']['chunk_text']}")
    print("---")

ID: chunk_451
Сходство: 0.41920352
Текст: 
7	Is Starvest plc’s business described as inherently low risk?	
No

---
ID: chunk_452
Сходство: 0.414533108
Текст: 
8	Does Starvest plc engage with stakeholders on a regular basis?	
Yes

---
ID: chunk_447
Сходство: 0.368950784
Текст: 
3	Is the company Starvest plc involved in any operating activities beyond its investment holdings?	
No

---
ID: chunk_448
Сходство: 0.349774748
Текст: 
4	Did Starvest plc make any charitable or political donations during the year?	
No

---
ID: chunk_454
Сходство: 0.327684224
Текст: 
10	Аre all Directors of Starvest plc required to attend board meetings at least quarterly?	
Yes

---


In [26]:
new_keys = {
    -2: -2,
    -1: -1,
    0: 0,
    1: 3,
    2: 4,
    3: 1,
    4: 2,
    5: 5,
    6: 6,
    7: 7,
    8: 8
}
topics = {
#    -2: 'a company from companies',
    -1: "Высшее образование",
    0: "Французский бульдог",
    1: "rmr (red_mad_robot)",
    2: "Telegram",
    3: "Уход за кожей",
    4: "СП35.13330.2010",
    5: "Michelin",
    6: "Трудовой кодекс РФ",
    7: "World Class",
    8: "Оплата услуг"
}
categories = {
    -2: "companies",
    -1: "Социальное обеспечение",
    0: "Собаководство",
    1: "companies",
    2: "companies",
    3: "Косметика",
    4: "СНиП",
    5: "companies",
    6: "Законодательство",
    7: "companies",
    8: "Оплата услуг"
}
companies = [
    'Starvest plc',
    'Brave Bison',
    'Rectifier Technologies Ltd'
]

keys = list(topics.keys())  # Предполагаем, что все три словаря имеют одинаковые ключи

for key in new_keys:
    if (key == -2):
        for company in companies:
            topic_name = company
            category = categories[new_keys[key]]
            print(f"Topic id: {[new_keys[key]]}, Topic: {topic_name}, Category: {category}")
    else:
        topic_name = topics[new_keys[key]]
        category = categories[new_keys[key]]
        print(f"Topic id: {[new_keys[key]]}, Topic: {topic_name}, Category: {category}")


Topic id: [-2], Topic: Starvest plc, Category: companies
Topic id: [-2], Topic: Brave Bison, Category: companies
Topic id: [-2], Topic: Rectifier Technologies Ltd, Category: companies
Topic id: [-1], Topic: Высшее образование, Category: Социальное обеспечение
Topic id: [0], Topic: Французский бульдог, Category: Собаководство
Topic id: [3], Topic: Уход за кожей, Category: Косметика
Topic id: [4], Topic: СП35.13330.2010, Category: СНиП
Topic id: [1], Topic: rmr (red_mad_robot), Category: companies
Topic id: [2], Topic: Telegram, Category: companies
Topic id: [5], Topic: Michelin, Category: companies
Topic id: [6], Topic: Трудовой кодекс РФ, Category: Законодательство
Topic id: [7], Topic: World Class, Category: companies
Topic id: [8], Topic: Оплата услуг, Category: Оплата услуг
